In [9]:
#Import functions
import aaCounter as a
import theoreticalLabellingFunctions as t

In [10]:
#Find lysines in full proteome
#Full proteome (would need to replace filepath in quotes to use a different file.)
lysines = a.aminoacidDetect("NbD_Proteome.fasta", ["K"])
print(lysines)
#Returns: 2122374

{'K': 2122374}


In [11]:
#Example mock file
exampleaas = a.aminoacidDetect("Mock_NbD_Proteome.fasta", ['K', 'R'])
print(exampleaas)
#In example, we have 124 lysines

{'K': 124, 'R': 96}


Potential peptides

example seq
MSALHTSGTWELVPLPSGKTTVGGRWVYAVKVGQDGKVDRLKARLVAKGYTQIFGLDYSD

expected peptides:
MSALHTSGTWELVPLPSGKTTVGGRWVYAVKVGQDGKVDRLKARLVAKGYTQIFGLDYSD  
MSALHTSGTWELVPLPSGK  
MSALHTSGTWELVPLPSGKTTVGGR  
MSALHTSGTWELVPLPSGKTTVGGRWVYAVK  
TTVGGR  
TTVGGRWVYAVK  
TTVGGRWVYAVKVGQDGK  
WVYAVK  
WVYAVKVGQDGK  
WVYAVKVGQDGKVDRLKAR  
VGQDGK  
VGQDGKVDR  
VGQDGKVDRLK  
VDR  
VDRLK  
VDRLKAR  
LK  
LKAR  
LKARLVAK  
AR  
ARLVAK  
ARLVAKGYTQIFGLDYSD  
LVAK  
LVAKGYTQIFGLDYSD  
GYTQIFGLDYSD  

In [3]:
exampleSequence = 'MSALHTSGTWELVPLPSGKTTVGGRWVYAVKVGQDGKVDRLKARLVAKGYTQIFGLDYSD'

In [9]:
# within the secretome (subset of proteome) calculate all potential peptides
## Didn't use this function in the end ##
def cutSiteDistances(proteinSequence, cutSites = ['R', 'K']):
    """
    Determines relative distances of cut sites in a protein sequence.

    params:
        proteinSequence: the protein sequence of interest (str)
        cutSites: list of amino acids at which the enzyme cuts (list of str)

    returns:
        cutSiteDistances: list of distances of cut sites relative to the previous cut site.
        
     """
    #This will be a list of all cut sites in the sequence, with start 0. start sites of peptides will iterate through this list.
    cutSiteDistances = [0]

    #Iterate through the sequence and collect all cut site positions
    cutIterator = 0
    for index, aa in enumerate(proteinSequence):
        if aa in cutSites:
            distanceFromPrevious = index - sum(cutSiteDistances)
            cutSiteDistances.append(distanceFromPrevious)
            cutIterator+=1

    return cutSiteDistances

In [5]:
# within the secretome (subset of proteome) calculate all potential peptides
def cutSiteSearchRegions(proteinSequence, cutsites = ['R', 'K']):
    """
    Determines relative distances of cut sites in a protein sequence.

    params:
        proteinSequence: the protein sequence of interest (str)
        cutSites: list of amino acids at which the enzyme cuts (list of str)

    returns:
        cutSiteDistances: list of distances of cut sites relative to the previous cut site.
        
     """
    #This will be a list of all cut sites in the sequence, with start 0. start sites of peptides will iterate through this list.
    cutSitePositions = []

    #Iterate through the sequence and collect all cut site positions
    for index, aa in enumerate(proteinSequence):
        if aa in cutsites:
            cutSitePositions.append(index)
        
    searchRegions = [proteinSequence] #starting region will be the whole protein (starts at 0)
    for cutSite in cutSitePositions:
            searchRegions.append(proteinSequence[cutSite+1:]) #note that this is +1 as the cut peptide itself should be left of the cut site
            
    return searchRegions

In [12]:
searchRegions =cutSiteSearchRegions(exampleSequence)
print(searchRegions, len(searchRegions))
searchRegionsR = cutSiteSearchRegions(exampleSequence, cutsites = ['R'])
print(searchRegionsR, len(searchRegionsR))
#Search regions are as expected, and are reduced when all lysines labelled (from 9 to 4)

['MSALHTSGTWELVPLPSGKTTVGGRWVYAVKVGQDGKVDRLKARLVAKGYTQIFGLDYSD', 'TTVGGRWVYAVKVGQDGKVDRLKARLVAKGYTQIFGLDYSD', 'WVYAVKVGQDGKVDRLKARLVAKGYTQIFGLDYSD', 'VGQDGKVDRLKARLVAKGYTQIFGLDYSD', 'VDRLKARLVAKGYTQIFGLDYSD', 'LKARLVAKGYTQIFGLDYSD', 'ARLVAKGYTQIFGLDYSD', 'LVAKGYTQIFGLDYSD', 'GYTQIFGLDYSD'] 9
['MSALHTSGTWELVPLPSGKTTVGGRWVYAVKVGQDGKVDRLKARLVAKGYTQIFGLDYSD', 'WVYAVKVGQDGKVDRLKARLVAKGYTQIFGLDYSD', 'LKARLVAKGYTQIFGLDYSD', 'LVAKGYTQIFGLDYSD'] 4


In [14]:
# within the secretome (subset of proteome) calculate all potential peptides

def potentialPeptides(proteinSequence, cutSites = ['R', 'K'], missedcleavagemax = 2):
    """
    Calculates all potential peptides for a protein sequence given the amino acids at which the enzyme cuts.

    params:
        proteinSequence: string of the protein sequenc of interest.
        cutSites: list of strings of amino acids at which the enzyme cuts. By default, this is set for trypsin.
    
    output:
        allPeptidesList: all potential peptides for the given inputs.
    """
    #Start list for the output - all potential peptides for this protein
    allPeptidesList = []

    #Counter for missed cleavages (allows for two)

    searchRegions = cutSiteSearchRegions(proteinSequence, cutsites = cutSites)

    #Iterate through the string and add potential peptides according to allowed missed cleavages.
    for searchRegion in searchRegions:
        peptideCount = 0
        for index, aa in enumerate(searchRegion):
        #If cut site is encountered, save sequence
            if aa in cutSites:
                stopaa = index 
                allPeptidesList.append(searchRegion[:stopaa+1]) #again to keep the cut protein on the left side
                peptideCount +=1
                #Once we've accounted for two missed cleavages, stop iterating and move to the next start site
                if peptideCount == missedcleavagemax+1: #missed cleavage allowance +1 = expected number of peptides output
                    break
            elif index == len(searchRegion)-1:
                allPeptidesList.append(searchRegion[:index+1])
                break                
        
    #Remove peptides which are out of range from the full list
    #allPeptidesList = inRangePeptideCalc(allPeptidesList, minSize = 5, maxSize = 10) hashed out for now as the min/max is nonsensical

    allPeptidesLength = len(allPeptidesList)
    
    return allPeptidesLength


In [15]:
#Unlabelled case
allPeptidesList = potentialPeptides(exampleSequence)
print(allPeptidesList)
#This works, returns all expected peptides as listed above

#Labelled case
allPeptidesListR = potentialPeptides(exampleSequence, cutSites = ["R"])
print(allPeptidesListR)
#This also prints as expected, this case has longer and fewer peptides


9
['MSALHTSGTWELVPLPSGK', 'MSALHTSGTWELVPLPSGKTTVGGR', 'MSALHTSGTWELVPLPSGKTTVGGRWVYAVK', 'TTVGGR', 'TTVGGRWVYAVK', 'TTVGGRWVYAVKVGQDGK', 'WVYAVK', 'WVYAVKVGQDGK', 'WVYAVKVGQDGKVDR', 'VGQDGK', 'VGQDGKVDR', 'VGQDGKVDRLK', 'VDR', 'VDRLK', 'VDRLKAR', 'LK', 'LKAR', 'LKARLVAK', 'AR', 'ARLVAK', 'ARLVAKGYTQIFGLDYSD', 'LVAK', 'LVAKGYTQIFGLDYSD', 'GYTQIFGLDYSD']
24
4
['MSALHTSGTWELVPLPSGKTTVGGR', 'MSALHTSGTWELVPLPSGKTTVGGRWVYAVKVGQDGKVDR', 'MSALHTSGTWELVPLPSGKTTVGGRWVYAVKVGQDGKVDRLKAR', 'WVYAVKVGQDGKVDR', 'WVYAVKVGQDGKVDRLKAR', 'WVYAVKVGQDGKVDRLKARLVAKGYTQIFGLDYSD', 'LKAR', 'LKARLVAKGYTQIFGLDYSD', 'LVAKGYTQIFGLDYSD']
9


In [160]:
def inRangePeptideCalc(peptideList, minSize = 5, maxSize = 10):
    """
    Determine the proportion of peptides that are undetectable given a size range.count

    params:
        peptideList: peptides to be considered
        minSize: minimum detectable peptide size.
        maxSize: maximum detectable peptide size.

    output:
        detectableRatio: ratio of peptides that are in range/total
        inRangePeptides: truncated peptideList for only proteins within the range
    """
    #note that if the min/max needs to be in kDa, need a kDa calculator function here

    inRangePeptides = []

    for peptide in peptideList:
        if len(peptide) > minSize and len(peptide) < maxSize:
            inRangePeptides.append(peptide)

    detectableRatio = len(inRangePeptides)/len(peptideList)

    return detectableRatio, inRangePeptides


In [161]:
detectableRatio, inRangePeptides = inRangePeptideCalc(allPeptidesList, minSize = 5, maxSize = 10)
print(detectableRatio, inRangePeptides)
#works

0.2916666666666667 ['TTVGGR', 'WVYAVK', 'VGQDGK', 'VGQDGKVDR', 'VDRLKAR', 'LKARLVAK', 'ARLVAK']


In [164]:
def returnAllPeptideCounts(proteinsfasta):
    """
    """
    from Bio import SeqIO

    totalUnlabelled = 0
    totalLabelled = 0

    #Iterate through sequences in the fasta file 
    for fasta in SeqIO.parse(proteinsfasta, "fasta"):
        sequence = fasta.seq

        totalUnlabelled += potentialPeptides(sequence, cutSites = ['R', 'K'])
        totalLabelled += potentialPeptides(sequence, cutSites = ['R'])
        
    return totalUnlabelled, totalLabelled


In [166]:
totalUnlabelled, totalLabelled = returnAllPeptideCounts("Mock_NbD_Proteome.fasta")
print(totalUnlabelled)
print(totalLabelled)


657
288
